In [13]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="unauthorized")
project = rf.workspace("yash-khurana-qbt8e").project("023948023984")
version = project.version(1)
dataset = version.download("yolov11")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 2.9 MB/s  0:00:18m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.3/892.3 kB 1.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 904.5 kB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 1.5 MB/s  0:00:03 eta 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 10.0.0
    Uninstalling Pillow-10.0.0:
      Successfully uninstalled Pillow-10.0.0
  Attempting uninstall: idnam╸━━━━━━━━━━━━━━━━━━━━━━ 4/9 [opencv-python-headless]
    Found existing installation: idna 3.10━━━━━━━━━━━━━━━━━━━━ 4/9 [opencv-python-headless]
    Uninstalling idna-3.10:91m╸━━━━━━━━━━━━━━━━━━━━━━ 4/9 [opencv-python-headless]
      Successfully uninstalled idna-3.10━━━━━━━━━━━━━━━━━━━━━━ 4/9 [opencv-python-headless]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [roboflow]6/9 [pi-heif]n-headless]


RuntimeError: {
    "error": {
        "message": "This API key does not exist (or has been revoked).",
        "status": 401,
        "type": "OAuthException",
        "hint": "You may retrieve your API key via the Roboflow Dashboard. Go to Account > Roboflow Keys to retrieve yours.",
        "key": "unauthorized"
    }
}

In [2]:
!pip install ultralytics opencv-python numpy


  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached polars-1.36.1-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached polars_runtime_32-1.36.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.3 MB/s eta 0:00:00a 0:00:01
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

In [ ]:
def detect_wheels(car_crop, frame, wheel_model, x1, x2, y1, y2):
    wheel_results = wheel_model.predict(car_crop, verbose=False, conf=0.25)
                
    wheels_list = []
    
    for w_result in wheel_results:
        w_boxes = w_result.boxes.xyxy.cpu().numpy()
        
        for w_box in w_boxes:
            wx1, wy1, wx2, wy2 = map(int, w_box)
            
            global_wx1 = x1 + wx1
            global_wy1 = y1 + wy1
            global_wx2 = x1 + wx2
            global_wy2 = y1 + wy2
            
            wheels_list.append([global_wx1, global_wy1, global_wx2, global_wy2])

            # Рисуем колеса (зеленым)
            cv2.rectangle(frame, (global_wx1, global_wy1), (global_wx2, global_wy2), (0, 255, 0), 2)
    
    return wheels_list


def process_video_traffic(input_video_path, output_video_path, wheel_model_path=None):
    car_model = YOLO('yolov8n.pt') 
    if wheel_model_path:
        wheel_model = YOLO(wheel_model_path)
    else:
        print("Внимание: Путь к модели колес не указан. Используется стандартная модель (результат детекции колес будет некорректным без дообучения).")
        wheel_model = YOLO('yolov8n.pt') 

    # Классы COCO, относящиеся к транспорту (2: car, 5: bus, 7: truck)
    vehicle_classes = [2, 5, 7] 

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Ошибка открытия видео")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_count = 0
    frames_data = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        frame_data = []

        results = car_model.track(frame, persist=True, classes=vehicle_classes, verbose=False)
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.int().cpu().numpy()
            
            for box, track_id in zip(boxes, track_ids):
                x1, y1, x2, y2 = map(int, box)
                
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(width, x2), min(height, y2)

                car_crop = frame[y1:y2, x1:x2]
                if car_crop.size == 0:
                    continue

                wheels_list = detect_wheels(car_crop, frame, wheel_model, x1, x2, y1, y2)

                car_info = {
                    "tracking_id": int(track_id),
                    "car_bbox": [x1, y1, x2, y2],
                    "wheels_bboxes": wheels_list
                }
                frame_data.append(car_info)
                

                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        print(f"Кадр {frame_count}: Обнаружено {len(frame_data)} машин.")
        frames_data.append({"frame": frame_count, "data": frame_data})
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return frames_data


In [27]:
if __name__ == "__main__":
    # Укажите путь к видео и путь к модели колес (если есть)
    data = process_video_traffic(
        input_video_path="./videos/railway_crash.mp4", 
        output_video_path="./results/output8.mp4",
        wheel_model_path="./models/wheels_yolov11.pt" # Раскомментируйте, если есть обученная модель
    )

Кадр 1: Обнаружено 1 машин.
Кадр 2: Обнаружено 1 машин.
Кадр 3: Обнаружено 1 машин.
Кадр 4: Обнаружено 1 машин.
Кадр 5: Обнаружено 1 машин.
Кадр 6: Обнаружено 1 машин.
Кадр 7: Обнаружено 0 машин.
Кадр 8: Обнаружено 0 машин.
Кадр 9: Обнаружено 0 машин.
Кадр 10: Обнаружено 0 машин.
Кадр 11: Обнаружено 0 машин.
Кадр 12: Обнаружено 0 машин.
Кадр 13: Обнаружено 0 машин.
Кадр 14: Обнаружено 0 машин.
Кадр 15: Обнаружено 0 машин.
Кадр 16: Обнаружено 0 машин.
Кадр 17: Обнаружено 0 машин.
Кадр 18: Обнаружено 0 машин.
Кадр 19: Обнаружено 0 машин.
Кадр 20: Обнаружено 0 машин.
Кадр 21: Обнаружено 0 машин.
Кадр 22: Обнаружено 0 машин.
Кадр 23: Обнаружено 0 машин.
Кадр 24: Обнаружено 0 машин.
Кадр 25: Обнаружено 0 машин.
Кадр 26: Обнаружено 1 машин.
Кадр 27: Обнаружено 1 машин.
Кадр 28: Обнаружено 1 машин.
Кадр 29: Обнаружено 1 машин.
Кадр 30: Обнаружено 1 машин.
Кадр 31: Обнаружено 1 машин.
Кадр 32: Обнаружено 1 машин.
Кадр 33: Обнаружено 1 машин.
Кадр 34: Обнаружено 1 машин.
Кадр 35: Обнаружено 1 м

In [ ]:
data[32]

{'frame': 331,
 'data': [{'tracking_id': 16,
   'car_bbox': [276, 135, 719, 531],
   'wheels_bboxes': None}]}

TypeError: 'NoneType' object is not subscriptable